In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import cv2
import torch
import numpy as np

# Load pre-trained YOLOv5 model (for person detection)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model.eval()

def detect_person_yolo(frame):
    """
    Use YOLO to detect a person in the frame and return the bounding box.
    """
    results = model(frame)
    # Extract the bounding boxes for persons (class id 0 is for 'person' in COCO dataset)
    person_boxes = [box for box in results.xyxy[0] if int(box[5]) == 0]
    
    if person_boxes:
        # Take the first detected person
        person_box = person_boxes[0].cpu().numpy()
        x1, y1, x2, y2 = map(int, person_box[:4])
        return (x1, y1, x2, y2), person_box[4]  # Returning bounding box and confidence
    return None, None

def segment_and_color_person(frame, bbox):
    """
    Highlight the detected person in the frame using a colored mask.
    """
    x1, y1, x2, y2 = bbox

    # Create a mask for the person
    mask = np.zeros_like(frame, dtype=np.uint8)
    
    # Set the mask to yellow in the bounding box area
    mask[y1:y2, x1:x2] = (0, 255, 255)  # Yellow color (BGR format)

    # Apply mask to the frame by blending the original frame and the mask
    segmented_frame = cv2.addWeighted(frame, 1, mask, 0.5, 0)

    return segmented_frame

def segment_and_save_video(video_path, output_path):
    """
    Detect and segment persons in a video and save the processed video.
    """
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Define video writer for output
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for output video
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Detect the person in the frame
        bbox, confidence = detect_person_yolo(frame)
        if bbox:
            # Highlight the detected person with a colored mask
            frame = segment_and_color_person(frame, bbox)
            
            # Draw bounding box around the person
            x1, y1, x2, y2 = bbox
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green bounding box
            
            # Add confidence score text
            cv2.putText(frame, f"Person: {confidence:.2f}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        # Write the modified frame (with the person segmented and colored) to the output video
        out.write(frame)
    
    cap.release()
    out.release()
    print(f"Processed video saved at: {output_path}")

# Example usage
video_path = '/kaggle/input/shop-dataset/Shop DataSet/shop lifters/shop_lifter_10.mp4'  # Path to the input video
output_path = 'output_segmented_colored_video.mp4'  # Path to save the segmented video
segment_and_save_video(video_path, output_path)
